In [3]:
import numpy as np
import gensim
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json.gz", lines = True) # It means that 1 line is 1 json object
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [13]:
df.shape

(194439, 9)

In [15]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [18]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")
# Its tokenizing sentences

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [19]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text # Every object is a list and list have tokenized words

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [20]:
model = gensim.models.Word2Vec(
    window = 10, # It means 10 before target word and 10 after target word | There is no rule its space for experiments with this number 
    min_count = 2, # Now it don`t use 1 word sentence, at least 2 words
    workers = 4 # How many cpu`s threads to use
)

In [21]:
model.build_vocab(review_text, # It will build unique list of words 
                  progress_per = 1000) # It will show me update after 1000 of words

In [23]:
model.epochs # Its default value of epochs

5

In [24]:
model.corpus_count

194439

In [25]:
model.train(review_text, total_examples = model.corpus_count, epochs = model.epochs)

(61503377, 83868975)

In [30]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [33]:
model.wv.most_similar("bad") # It show me similar words with similarity score | good and bad are antonyms 

[('shabby', 0.6604461669921875),
 ('terrible', 0.6539217829704285),
 ('good', 0.5913695693016052),
 ('horrible', 0.5858848094940186),
 ('crappy', 0.528866171836853),
 ('ok', 0.5239594578742981),
 ('disappointing', 0.5182961225509644),
 ('legit', 0.51675945520401),
 ('awful', 0.5135593414306641),
 ('okay', 0.5133675336837769)]

In [35]:
model.wv.similarity(w1 = "cheap", w2 = "inexpensive") # Its cosine similarity between this two words

0.52607566

In [36]:
model.wv.similarity(w1 = "great", w2 = "good")

0.7858423

In [37]:
model.wv.similarity(w1 = "great", w2 = "product")

-0.028002005